# Prediction

Deployment of kserve will take few minutes to comes to ready state. In just a short span of time, it will transition
into a ready state and you'll be in a state where you can invoke the service you deployed. You can check the status of
your deployment by navigating to the Kubeflow Endopoints UI.

In [ ]:
import os
import requests

In [ ]:
DOMAIN_NAME = "svc.cluster.local"  # change this to your domain for external access
NAMESPACE = os.getenv("USER")
DEPLOYMENT_NAME = "bike-sharing"
MODEL_NAME = DEPLOYMENT_NAME
SVC = f"{DEPLOYMENT_NAME}-predictor-default.{NAMESPACE}.{DOMAIN_NAME}"
URL = f"https://{SVC}/v2/models/{MODEL_NAME}/infer"

print(URL)

In [ ]:
names = ["season", "year", "month", "hour_of_day", "is_holiday", "weekday",
         "is_workingday", "weather_situation", "temperature",
         "feels_like_temperature", "humidity", "windspeed"]

input_data = [
    [1, 2, 1, 0, 0, 6, 0, 1, 0.24, 0.2879, 0.81, 0.0000],
    [1, 5, 1, 0, 0, 6, 1, 1, 0.24, 0.2879, 0.81, 0.0000]
]

inputs = {
  "inputs": [
    {
      "name": "ndarray",
      "shape": [2, 12],
      "datatype": "FP32",
      "data": input_data
    }
  ]
}

headers = {"Authorization": f"Bearer {os.getenv('AUTH_TOKEN')}"}
response = requests.post(URL, json=inputs, headers=headers, verify=False)

print(f"Rented Bikes Per Hour: {response.json()['outputs'][0]['data']}")